# NLTK Punkt 

In [198]:
%pip install nltk

  Using cached https://files.pythonhosted.org/packages/73/56/90178929712ce427ebad179f8dc46c8deef4e89d4c853092bee1efd57d05/nltk-3.4.1.zip
  Stored in directory: /Users/ozgedincsoy/Library/Caches/pip/wheels/97/8a/10/d646015f33c525688e91986c4544c68019b19a473cb33d3b55
Successfully built nltk
Note: you may need to restart the kernel to use updated packages.
  Could not find a version that satisfies the requirement pickle (from versions: )
No matching distribution found for pickle
Note: you may need to restart the kernel to use updated packages.


In [218]:
import nltk
import pickle
import json
from nltk.tokenize import sent_tokenize

sent_detec = nltk.data.load('tokenizers/punkt/turkish.pickle')

f = open("derlem.txt", encoding="utf-16", errors='ignore')
corpus = f.read()
par_list = corpus.split("\n\n")

par_dict = {}

for par in par_list:
    start = par.index(" ")
    num = par[:start]
    key = int(num)
    paragraph = par[start+1:]
    sentences = sent_detec.tokenize(par[start+1:])
    lemmas = []
    for sentence in sentences:
        result = analyze(sentence)
        l = {}
        for a in result.results:
            if a.best.pos != 'Punc':
                element = a.best.lemmas[0]
                if a.best.lemmas[0] == 'UNK':
                    element = a.token
                count = l.get(element,0)
                count += 1
                l[element] = count
        lemmas.append(l)
                
    par_dict[key] = { "paragraph" : paragraph, "sentences" : sentences, "lemmas" : lemmas}

with open('parToSentences.pickle', 'wb') as handle:
    pickle.dump(par_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [217]:
aa = 0
for e in par_dict:
    print(par_dict[e])
    aa += 1
    if aa == 5:
        break

{'paragraph': 'Madenler, yaşamımızın birçok alanında çeşitli amaçlarla işleyerek kullandığımız doğal kaynaklardır. İnşaat, otomotiv, beyaz eşya, mutfak eşyaları, elektrikli ve elektronik eşya üretimi gibi birçok alanda madenler kullanılmaktadır.', 'sentences': ['Madenler, yaşamımızın birçok alanında çeşitli amaçlarla işleyerek kullandığımız doğal kaynaklardır.', 'İnşaat, otomotiv, beyaz eşya, mutfak eşyaları, elektrikli ve elektronik eşya üretimi gibi birçok alanda madenler kullanılmaktadır.'], 'lemmas': [{'maden': 1, 'yaşam': 1, 'birçok': 1, 'alan': 1, 'çeşit': 1, 'amaç': 1, 'işle': 1, 'kullan': 1, 'doğal': 1, 'kaynakla': 1}, {'inşaat': 1, 'otomotiv': 1, 'beyaz': 1, 'eşya': 3, 'mutfak': 1, 'elektrik': 1, 've': 1, 'elektronik': 1, 'üretim': 1, 'gibi': 1, 'birçok': 1, 'alan': 1, 'maden': 1, 'kullan': 1}]}
{'paragraph': 'Türkiye, maden çeşidi bakımından zengin bir ülkedir. Bu maden yataklarının önemli bir kısmı işlenmektedir. Bir maden yatağı işletmeye açılırken o madenin toplam miktarı 

# Docker Installation for Zemberek

brew cask install docker

open --background -a Docker

sudo docker run -d --rm -p 6789:6789 --name zemberek-grpc ryts/zemberek-grpc

# Zemberek 

In [2]:
%pip install zemberek-grpc

Note: you may need to restart the kernel to use updated packages.


In [202]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import sys

import grpc

import zemberek_grpc.language_id_pb2 as z_langid
import zemberek_grpc.language_id_pb2_grpc as z_langid_g
import zemberek_grpc.normalization_pb2 as z_normalization
import zemberek_grpc.normalization_pb2_grpc as z_normalization_g
import zemberek_grpc.preprocess_pb2 as z_preprocess
import zemberek_grpc.preprocess_pb2_grpc as z_preprocess_g
import zemberek_grpc.morphology_pb2 as z_morphology
import zemberek_grpc.morphology_pb2_grpc as z_morphology_g

channel = grpc.insecure_channel('localhost:6789')

langid_stub = z_langid_g.LanguageIdServiceStub(channel)
normalization_stub = z_normalization_g.NormalizationServiceStub(channel)
preprocess_stub = z_preprocess_g.PreprocessingServiceStub(channel)
morphology_stub = z_morphology_g.MorphologyServiceStub(channel)

def find_lang_id(i):
    response = langid_stub.Detect(z_langid.LanguageIdRequest(input=i))
    return response.langId

def tokenize(i):
    response = preprocess_stub.Tokenize(z_preprocess.TokenizationRequest(input=i))
    return response.tokens

def normalize(i):
    response = normalization_stub.Normalize(z_normalization.NormalizationRequest(input=i))
    return response

def analyze(i):
    response = morphology_stub.AnalyzeSentence(z_morphology.SentenceAnalysisRequest(input=i))
    return response;

def fix_decode(text):
    """Pass decode."""
    if sys.version_info < (3, 0):
        return text.decode('utf-8')
    else:
        return text

In [23]:
import sys
import random
import os
import collections
from pathlib import Path

cwd = os.getcwd()

GROUP_FILE_NAME = '2.txt'

# _, test_questions_path, task1_pred_path, task2_pred_path = sys.argv

In [16]:
# Since we do not take the argument from command line yet
task1_pred_path = cwd + "/task1_predictions"

# Lemmatizer from @akoksal

In [17]:
import re
import json
import pickle
import sys

class Lemmatizer:
    def __init__(self):
        try:
            with open('revisedDict.pkl', 'rb') as f:
                self.revisedDict = pickle.load(f)
        except IOError:
            print("Please run trainLexicon.py to generate revisedDict.pkl file")

    def check(self, root, suffix, guess, action):
        if action == "unsuz yumusamasi":
            return len(suffix)>0 and suffix[0] in ["a","e","ı","i","o","ö","u","ü"] and self.checkSuffixValidation(suffix)[0]
        if action == "unlu daralmasi":
            if guess=="demek" and self.checkSuffixValidation(suffix)[0]:
                return True
            if guess=="yemek" and self.checkSuffixValidation(suffix)[0]:
                return True
            
            if suffix.startswith("yor"):
                lastVowel = ""
                for letter in reversed(guess[:-3]):
                    if letter in ["a","e","ı","i","o","ö","u","ü"]:
                        lastVowel = letter
                        break
                if lastVowel in ["a","e"] and self.checkSuffixValidation(suffix)[0]:
                    return True
            return False
        if action == "fiil" or action == "olumsuzluk eki":
            return self.checkSuffixValidation(suffix)[0] and not ((root.endswith("la") or (root.endswith("le"))) and suffix.startswith("r"))
        if action == "unlu dusmesi":
            count = 0
            for letter in guess:
                if letter in ["a","e","ı","i","o","ö","u","ü"]:
                    count+=1
                    lastVowel = letter
            if self.checkSuffixValidation(suffix)[0] and count==2 and (lastVowel in ["ı","i","u","ü"]) and (len(suffix)>0 and suffix[0] in ["a","e","ı","i","o","ö","u","ü"]):
                if lastVowel == "ı":
                    return suffix[0] in ["a","ı"]
                elif lastVowel == "i":
                    return suffix[0] in ["e","i"]
                elif lastVowel == "u":
                    return suffix[0] in ["a","u"]
                elif lastVowel == "ü":
                    return suffix[0] in ["e","ü"]
            return False
        return True
        
    def findPos(self, kelime):
        revisedDict = self.revisedDict
        l = []
        if "'" in kelime:
            l.append([kelime[:kelime.index("'")]+"_1","tirnaksiz",kelime])
        mid = []
        for i in range(len(kelime)):
            guess = kelime[:len(kelime)-i]
            suffix = kelime[len(kelime)-i:]
            ct = 1
            
            while guess+"_"+str(ct) in revisedDict:
                if self.check(guess, suffix, revisedDict[guess+"_"+str(ct)][1], revisedDict[guess+"_"+str(ct)][0]):
                    guessList = (revisedDict[guess+"_"+str(ct)])
                    while guessList[0] not in ["kok","fiil","olumsuzluk"]:
                        guessList = revisedDict[guessList[1]]
                    mid.append([guessList[1], revisedDict[guess+"_"+str(ct)][0],guess+"_"+str(ct)])
                ct = ct+1
                
        temp = []
        for kel in mid:
            kelime_kok = kel[0][:kel[0].index("_")]
            kelime_len = len(kelime_kok)
            if kelime_kok.endswith("mak") or kelime_kok.endswith("mek"):
                kelime_len -= 3
            not_inserted = True
            for index in range(len(temp)):
                temp_kelime = temp[index]
                temp_kelime_kok = temp_kelime[0][:temp_kelime[0].index("_")]
                temp_len = len(temp_kelime_kok)
                if temp_kelime_kok.endswith("mak") or temp_kelime_kok.endswith("mek"):
                    temp_len -= 3
                if(kelime_len>temp_len):
                    temp.insert(index,kel)
                    not_inserted = False
            if not_inserted:
                temp.append(kel)
        output = l+temp
        if len(output)==0:
            output.append([kelime+"_1","çaresiz",kelime+"_1",])
        return output

    def checkSuffixValidation(self, suff):
        suffixList = ["","a", "abil", "acağ", "acak", "alım", "ama", "an", "ar", "arak", "asın", "asınız", "ayım", "da", "dan", "de", "den", "dı", "dığ", "dık", "dıkça", "dır", "di", "diğ", "dik", "dikçe", "dir", "du", "duğ", "duk", "dukça", "dur", "dü", "düğ", "dük", "dükçe", "dür", "e", "ebil", "eceğ", "ecek", "elim", "eme", "en", "er", "erek", "esin", "esiniz", "eyim", "ı", "ıl", "ım", "ımız", "ın", "ınca", "ınız", "ıp", "ır", "ıyor", "ız", "i", "il", "im", "imiz", "in", "ince", "iniz", "ip", "ir", "iyor", "iz", "k", "ken", "la", "lar", "ları", "ların", "le", "ler", "leri", "lerin", "m", "ma", "madan", "mak", "maksızın", "makta", "maktansa", "malı", "maz", "me", "meden", "mek", "meksizin", "mekte", "mektense", "meli", "mez", "mı", "mış", "mız", "mi", "miş", "miz", "mu", "muş", "mü", "muz", "müş", "müz", "n", "nın", "nız", "nin", "niz", "nun", "nuz", "nün", "nüz", "r", "sa", "se", "sı", "sın", "sınız", "sınlar", "si", "sin", "siniz", "sinler", "su", "sun", "sunlar", "sunuz", "sü", "sün", "sünler", "sünüz", "ta", "tan", "te", "ten", "tı", "tığ", "tık", "tıkça", "tır", "ti", "tiğ", "tik", "tikçe", "tir", "tu", "tuğ", "tuk", "tukça", "tur", "tü", "tüğ", "tük", "tükçe", "tür", "u", "ul", "um", "umuz", "un", "unca", "unuz", "up", "ur", "uyor", "uz", "ü", "ül", "ün", "üm", "ümüz", "ünce", "ünüz", "üp", "ür", "üyor", "üz", "ya", "yabil", "yacağ", "yacak", "yalım", "yama", "yan", "yarak", "yasın", "yasınız", "yayım", "ydı", "ydi", "ydu", "ydü", "ye", "yebil", "yeceğ", "yecek", "yelim", "yeme", "yen", "yerek", "yesin", "yesiniz", "yeyim", "yı", "yım", "yın", "yınca", "yınız", "yıp", "yız", "yi", "yim", "yin", "yince", "yiniz", "yip", "yiz", "yken", "yla", "yle", "ymış", "ymiş", "ymuş", "ymüş", "yor", "ysa", "yse", "yu", "yum", "yun", "yunca", "yunuz", "yup", "yü", "yuz", "yüm", "yün", "yünce", "yünüz", "yüp", "yüz"]
        validList = []
        if suff in suffixList:
            validList.append(suff)
        for ind in range(1,len(suff)):
            if(suff[:ind] in suffixList):
                cont, contList = self.checkSuffixValidation(suff[ind:])
                if cont:
                    contList = [suff[:ind]+"+"+l for l in contList]
                    validList = validList+contList
        return len(validList)>0,validList

    def lemmatize(self, text):
        dc  = []
        for i in text.split(' '):
            findings = self.findPos(i.lower())
            word = findings[0][0]
            dc.append(findings[0][0][:-2])
    
        return dc

# Helper Functions

In [18]:
import re
from string import punctuation

def containArr(arr1, arr2):
    """returns true if at least one element from arr1 is contained by arr2"""
    for element in arr1:
        if element in arr2:
            return True
    return False

def remove_punctuation(text):
    """
    Removes punctuations from given text
    """
    # Replace punctuation with space instead of remove it for hand-to-mouth, six-week-old, euro-certificate
    return text.translate(str.maketrans(punctuation, ' ' * len(punctuation)))


def remove_extra_whitespaces(text):
    """
    Removes extra whitespaces from given text such as multiple adjencent space
    """
    return re.sub(r'\s+', ' ', text).strip()


def replace_capital_i(text):
    """
    Replaces İ with i since it gave error on @akoksal 's script.
    """
    return text.replace("İ", "i")


def normalize(text):
    """
    Normalizes the text.
    """
    return remove_extra_whitespaces(remove_punctuation(replace_capital_i(text))).lower()


def tokenize(text):
    """
    Returns a word array from the given text.
    """
    foo = text.split(' ')
    return foo


def tokenize_lemmatize(text):
    """
    Returns a word array from the given text.
    """
    foo = text.split(' ')
    new_items = [lemmatizer.lemmatize(i)[0] for i in foo]
    return new_items


def find_sentence(que, par):
    """
    Returns the related sentence to the question in the paragraph.
    """
    sent = par
    return sent

# Read Database
## Read derlem.txt

In [20]:
import os, re

'''
Transfer DERLEM to a Dictionary.
'''
with open("derlem.txt", encoding="utf-16") as f:
    opened_file = f.readlines()

'''
The dictionary of the DERLEM.
{id: paragraph as string}
'''
file_dict = {}
for i, line in enumerate(opened_file):
    if line != "\n":
        '''
        The first number in the line is the Paragraph ID,
        assign the Paragraph ID as the key to the dictionary,
        and the rest of the paragraph to the value.
        '''
        file_dict[int(line.split(' ', 1)[0])] = re.findall(" (.*)", line)[0]
    
# print(len(file_dict))

'''
The normalized dictionary of the DERLEM.
{id: array of tokens}
'''
norm_dict = file_dict.copy()
lemmatizer = Lemmatizer()
for i in norm_dict:
    norm_dict[i] = lemmatizer.lemmatize(normalize(file_dict[i]))
    
# print(norm_dict)

## Read soru_gruplari.txt

In [21]:
# Change the encoding of the input file to latin-1
file_content = ""
with open("soru_gruplari.txt", encoding="utf-16") as f:
    file_content = f.read()
# Read the file and append all of the lines
question_groups = file_content.split("\n\n")

In [24]:
question_groups_dict = {}
for group in question_groups:
    lines = group.split("\n")
    answer_id = ""
    group_dict = collections.defaultdict(dict)
    for line in lines:
        ID = re.findall("(.*):", line)[0]
        sentence = re.findall(": (.*)", line)[0]
        if ID[0] == 'S':  # if the sentence is a question
            group_dict["questions"][ID] = sentence
        elif ID[0] == 'C':  # if the sentence is the answer
            group_dict["answer"] = sentence
            answer_id = ID
        else:  # if the sentence is the id of related paragraph
            group_dict["paragraph"] = sentence
    question_groups_dict[answer_id] = group_dict

In [25]:
paragraphs_and_terms = {}
all_terms = set()
for paragraph_id in norm_dict:
    terms = norm_dict[paragraph_id]
    all_terms.update(terms)
    paragraphs_and_terms[paragraph_id] = terms
all_terms = list(all_terms)

In [26]:
# Compute term frequency for each term and paragraph
tf = {}
for paragraph_id in norm_dict:
    for term in norm_dict[paragraph_id]:
        key = (paragraph_id, term)
        tf[key] = norm_dict[paragraph_id].count(term)

In [27]:
# Compute document frequency for each term
df = {}
for term in all_terms:
    df[term] = sum([1 for paragraph_id in norm_dict if norm_dict[paragraph_id].count(term) > 0])

Tf.idf score for a given query (q) and the set of paragraphs (documents - d) is $\sum\limits_{t\in q\cap d} tf.idf_{t,d}$ where

tf score = $\begin{cases}
  1 + log_{10}tf_{t, d}, & \text{if $tf_{t, d} > 0$}\\
  0, & \text{otherwise}
\end{cases}  $

and

idf score = $log_{10}(N / df_{t})$

In [28]:
import math
import collections

# Compute tf-idf scores for each paragraph
weights = collections.defaultdict(list)
for paragraph_id in norm_dict:
    for term in all_terms:
        if((paragraph_id, term) in tf):
            weights[paragraph_id].append((1 + math.log10(tf[(paragraph_id, term)])) * math.log10(len(norm_dict)/df[term]))
        else:
            weights[paragraph_id].append(0)

## Predict paragraph

In [21]:
def predict_related_paragraph(question):
    lemmatizer = Lemmatizer()
    query_terms = lemmatizer.lemmatize(normalize(question))
    
    query_term_indices = [all_terms.index(term) for term in query_terms if term in all_terms]

    tf_idf = {}
    for paragraph_id in weights:
        tf_idf[paragraph_id] = sum([weight for index, weight in enumerate(weights[paragraph_id]) if index in query_term_indices])

    related_paragraphs = sorted(tf_idf.items(), key=lambda x: x[1], reverse=True)
    
    return related_paragraphs[0][0]

## Predict answer

In [195]:
def predict_answer(question):
    pass

## Write predicted results to files

In [23]:
related_paragraphs = []
answers = []
with open(cwd + '/test_questions.txt', 'r', encoding="utf-16") as f:
    questions = f.readlines()
    for question in questions:
        paragraph_id = predict_related_paragraph(question)
        answer = predict_answer(question)
        related_paragraphs.append(str(paragraph_id))
        answers.append(answer)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/dilrubareyyan/Codes/dolu-dolu-anadolu/test_questions.txt'

In [ ]:
# Uncomment the following line if you are running this for the first time
# os.mkdir(task1_pred_path)

with open(Path(task1_pred_path) / GROUP_FILE_NAME, 'w+', encoding='utf16') as f:
    f.write('\n'.join(related_paragraphs))

# TODO: complete task 2    
'''
with open(Path(task2_pred_path) / GROUP_FILE_NAME, 'w', encoding='utf16') as f:
    f.write('\n'.join(answers))
'''

# Extracting the Question from The Paragraph
Assuming that the right sentence from the right paragraph is given for the question.

In [192]:
def find_answer(question, answerSentence):
    analysis_result = analyze(answerSentence)
    answer_words = []
    answer_types = []
    a_dic = {}
    index = 0
    for a in analysis_result.results:
        best = a.best
        answer_words.append(best.lemmas[0])
        answer_types.append(best.pos)
        b = []
        b.append(index)
        b.append(a.token)
        b.append(best.lemmas[0])
        b.append(best.pos)
        index += 1
        a_dic[best.lemmas[0]] = b
        lemmas = ""
        for l in best.lemmas:
            lemmas = lemmas + " " + l
        #print("Word = " + a.token + ", Lemmas = " + lemmas + ", POS = [" + best.pos + "], Full Analysis = {" + best.analysis + "}")
        
    question_words = []
    question_types = {}
    q = analyze(question)
    print(a_dic)
    for a in q.results:
        best = a.best
        question_words.append(best.lemmas[0])
        question_types[best.lemmas[0]]=(best.pos)
        lemmas = ""
        for l in best.lemmas:
            lemmas = lemmas + " " + l
        print("Word = " + a.token + ", Lemmas = " + lemmas + ", POS = [" + best.pos + "], Full Analysis = {" + best.analysis + "}")
        
    l1 = (list(set(question_words)-set(answer_words)))    
    l2 = (list(set(answer_words)-set(question_words)))
    
    l2.sort(key=lambda x: a_dic[x][0], reverse=False)
    print(l2)
    print("\n--------------\n")
    
    res = []
    #print(l1)
    if containArr(["kaç","yıl","kadar","sene"], question_words):
        print("a")
        res = ([a_dic[x][1] for x in l2 if a_dic[x][3] == 'Num'])
    elif containArr(["bölge","ülke","kıta","il", "ilçe", "köy", "eyalet"], question_words):
        print("b")

        res = ([a_dic[x][1]  for x in l2 if a_dic[x][3] == 'Noun'])
    elif "nasıl" in question_words:
        print("c")

        res = ([a_dic[x][1]  for x in l2 if a_dic[x][3] == 'Adj' or a_dic[x][3] == 'Adv' or a_dic[x][3] == 'Verb' ])
    elif containArr(["ad","isim"], question_words): #bakılacak
        print("d")
        res = ([a_dic[x][1]  for x in l2 if a_dic[x][3] == 'Noun'])
    elif "kim" in question_words:
        print("e")
        res = ([a_dic[x][1]  for x in l2 if a_dic[x][3] == 'Noun'])
    elif containArr(["mı","mi","mu","mü"],question_words):
        if len([a_dic[x][1] for x in l2 if a_dic[x][3] == 'Verb']) != 0:
            print("f")
            res = ([a_dic[x][1] for x in answer_words if a_dic[x][3] == 'Verb'])
        else:
            print("g")
            res = ([a_dic[x][1]  for x in l2 if a_dic[x][3] =='Adv'])
    else:
        res = [a_dic[x][1] for x in l2 if (a_dic[x][3] != "Postp" and a_dic[x][3] != "Conj" and a_dic[x][3] != "Punc" and a_dic[x][3] != "Det")]
    print(res)

In [193]:
find_answer("istanbulun başkentini yazınız?", "istanbulun başkenti ankarada yer alır.")

{'istanbul': [0, 'istanbulun', 'istanbul', 'Noun'], 'başkent': [1, 'başkenti', 'başkent', 'Noun'], 'ankara': [2, 'ankarada', 'ankara', 'Noun'], 'yer': [3, 'yer', 'yer', 'Noun'], 'al': [4, 'alır', 'al', 'Verb'], '.': [5, '.', '.', 'Punc']}
Word = istanbulun, Lemmas =  istanbul, POS = [Noun], Full Analysis = {[İstanbul:Noun,Prop] istanbul:Noun+A3sg+un:Gen}
Word = başkentini, Lemmas =  başkent, POS = [Noun], Full Analysis = {[başkent:Noun] başkent:Noun+A3sg+i:P3sg+ni:Acc}
Word = yazınız, Lemmas =  yazı, POS = [Noun], Full Analysis = {[yazı:Noun] yazı:Noun+A3sg+nız:P2pl}
Word = ?, Lemmas =  ?, POS = [Punc], Full Analysis = {[?:Punc] ?:Punc}
['ankara', 'yer', 'al', '.']

--------------

['ankarada', 'yer', 'alır']
